## Congestion domain

This python notebook shows the basic usage of the unified planning library + social laws extension to describe and solve a simple problem.

[![Open In GitHub](https://img.shields.io/badge/see-Github-579aca?logo=github)](https:///github.com/aiplan4eu/unified-planning/blob/master/docs/notebooks/01-basic-example.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aiplan4eu/unified-planning/blob/master/docs/notebooks/01-basic-example.ipynb)

## Setup


### Imports

In [1]:
from up_social_laws.ma_problem_waitfor import MultiAgentProblemWithWaitfor
from unified_planning.shortcuts import *
from unified_planning.model.multi_agent import *
up.environment.get_environment().credits_stream = None

## Problem representation

In this example, we will model a very simple robot navigation problem.

### Types

The first thing to do is to introduce a "UserType" to model the concept of a location. It is possible to introduce as many types as needed; then, for each type we will define a set of objects of that type.  

In addition to `UserType`s we have three built-in types: `Bool`, `Real` and `Integer`.

In [2]:
problem = MultiAgentProblemWithWaitfor("congestion")

In [3]:
Location = UserType('Location')

### Fluents and constants

In [5]:
problem.ma_environment.clear_fluents()

# Location fluent
robot_at = Fluent('robot_at', BoolType(), l=Location)

# Free location fluent
free = Fluent('free', BoolType(), l=Location)
# free = problem.ma_environment.add_fluent('free',
#                                         BoolType(),
#                                         l=Location,
#                                         default_initial_value=True)
problem.ma_environment.add_fluent(free, default_initial_value=True)

# Adjacency fluent
connected = problem.ma_environment.add_fluent('connected', 
                                              BoolType(), 
                                              l_from=Location, 
                                              l_to=Location, 
                                              default_initial_value=False)

### Actions

In [6]:
move = InstantaneousAction('move', l_from=Location, l_to=Location)
l_from = move.parameter('l_from')
l_to = move.parameter('l_to')
move.add_precondition(connected(l_from, l_to))
move.add_precondition(free(l_to))
move.add_precondition(robot_at(l_from))
move.add_effect(robot_at(l_from), False)
move.add_effect(free(l_from), True)
move.add_effect(free(l_to), False)
move.add_effect(robot_at(l_to), True)

## Creating the problem

In [ ]:
# problem = unified_planning.model.Problem('congestion')
# problem.add_fluent(robot_at, default_initial_value=False)
# problem.add_fluent(connected, default_initial_value=False)
# problem.add_action(move)

### Adding locations

In [7]:
NLOC = 11
locations = [Object(f'l{i+1}', Location) for i in range(NLOC)]
problem.add_objects(locations)

### Location adjacency

Describing the location adjacency list as per [Overleaf document](https://www.overleaf.com/project/6485ea4fc610a58da86e2178).

![picture 0](https://i.imgur.com/zHkWzky.png)  

In [8]:
connections = [
    (1, 2), (2, 3), (3, 4),
    (1, 5), (2, 5), (3, 5), (4, 5),
    (1, 6), (2, 6), (3, 6), (4, 6),
    (5, 7), (6, 7),
    (7, 8), (7, 9), (7, 10), (7, 11),
    (8, 9), (9, 10), (10, 11)
]

for i, j in connections:
    problem.set_initial_value(connected(locations[i-1], locations[j-1]), True)
    print(f'{i} -> {j}')

1 -> 2
2 -> 3
3 -> 4
1 -> 5
2 -> 5
3 -> 5
4 -> 5
1 -> 6
2 -> 6
3 -> 6
4 -> 6
5 -> 7
6 -> 7
7 -> 8
7 -> 9
7 -> 10
7 -> 11
8 -> 9
9 -> 10
10 -> 11


### Agents

Now we're setting the agents. We need robots to be in locations `l1-l4`, and get to locations `l8-l11` in the end.

In [11]:
problem.clear_agents()

robots_number = 4
robots = []

for i in range(robots_number):
    robot = Agent(f'robot{i+1}', problem)
    robots.append(robot)
    robot.add_public_fluent(robot_at)
    # robot.add_public_fluent(free)
    robot.add_action(move)
    problem.add_agent(robot)

for index, robot in enumerate(robots):
    problem.set_initial_value(Dot(robot, robot_at(locations[index])), True)
    problem.set_initial_value(free(locations[index]), False)

### Final checks

Checking that our robots have been planted correctly:

In [ ]:
# for i, v in enumerate(robots):
#     print(Dot(v, robot_at(locations[i])))

And checking that our locations have been connected correctly:

In [12]:
# Prints a list of all the location connections sorted alphabetically
sorted([f'{key}: {value}' for key, value in problem.explicit_initial_values.items() if (key)])

['connected(l1, l1): false',
 'connected(l1, l10): false',
 'connected(l1, l11): false',
 'connected(l1, l2): true',
 'connected(l1, l3): false',
 'connected(l1, l4): false',
 'connected(l1, l5): true',
 'connected(l1, l6): true',
 'connected(l1, l7): false',
 'connected(l1, l8): false',
 'connected(l1, l9): false',
 'connected(l10, l1): false',
 'connected(l10, l10): false',
 'connected(l10, l11): true',
 'connected(l10, l2): false',
 'connected(l10, l3): false',
 'connected(l10, l4): false',
 'connected(l10, l5): false',
 'connected(l10, l6): false',
 'connected(l10, l7): false',
 'connected(l10, l8): false',
 'connected(l10, l9): false',
 'connected(l11, l1): false',
 'connected(l11, l10): false',
 'connected(l11, l11): false',
 'connected(l11, l2): false',
 'connected(l11, l3): false',
 'connected(l11, l4): false',
 'connected(l11, l5): false',
 'connected(l11, l6): false',
 'connected(l11, l7): false',
 'connected(l11, l8): false',
 'connected(l11, l9): false',
 'connected(l2, l1)

### Goals

In [35]:
# Sanity check for agents
for index, agent in enumerate(problem.agents):
    print(agent.name)

robot1
robot2
robot3
robot4


In [54]:
# print([method for method 
#        in dir(problem.agents[0]) 
#        if not method.startswith('_') 
#        and 'add' in method])

for method in dir(problem.agents[0]):
    if not method.startswith('_') and 'add' in method:
        print(method)

add_action
add_actions
add_fluent
add_fluents
add_private_fluent
add_private_fluents
add_public_fluent
add_public_fluents


Defining the goals for robots:

In [74]:
problem.clear_goals()

for index, robot in enumerate(problem.agents):
    current = problem.agent(robot.name)
    # https://unified-planning.readthedocs.io/en/latest/problem_representation.html#multiagent-example
    # problem.add_goal(Equals(Dot(current, robot_at), locations[index+7]))
    # The only way that seems to work
    problem.add_goal(Dot(current, robot_at(locations[index+7], True)))
    # A hacky way to add public goals (not working)
    # current.public_goals = robot_at(locations[index+7])
    # https://github.com/aiplan4eu/up-social-laws/blob/master/docs/notebook/Social_laws_usecase.ipynb
    # current.add_public_goal(robot_at(locations[index+7]))

UPExpressionDefinitionError: In FluentExp, fluent: robot_at has arity 1 but 2 parameters were passed.

Checking our goals had been set correctly:

In [65]:
problem.goals

[]

And printing the problem to check everything is as expected:

In [67]:
print (problem)

problem name = congestion

types = [Location]

environment fluents = [
  bool free[l=Location]
  bool connected[l_from=Location, l_to=Location]
]

agents = [
  Agent name = robot1

private fluents = [
]

public fluents = [
 bool robot_at[l=Location]
]

actions = [
 action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      free(l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      free(l_from) := true
      free(l_to) := false
      robot_at(l_to) := true
    ]
  }
]


  Agent name = robot2

private fluents = [
]

public fluents = [
 bool robot_at[l=Location]
]

actions = [
 action move(Location l_from, Location l_to) {
    preconditions = [
      connected(l_from, l_to)
      free(l_to)
      robot_at(l_from)
    ]
    effects = [
      robot_at(l_from) := false
      free(l_from) := true
      free(l_to) := false
      robot_at(l_to) := true
    ]
  }
]


  Agent name = robot3

private fluents = [
]

publ

## Robustness check

### Trying to compile to a SAP

In [69]:
from up_social_laws.single_agent_projection import SingleAgentProjection

for agent in problem.agents:
    projection = SingleAgentProjection(agent)
    print(projection)

AttributeError: MA_SINGLE_AGENT_PROJECTION

In [ ]:
from up_social_laws.robustness_checker import SocialLawRobustnessChecker

slrc = SocialLawRobustnessChecker()

result = slrc.is_single_agent_solvable(problem)

In [ ]:
from up_social_laws.synthesis import SocialLawGenerator, get_gbfs_social_law_generator

generator = get_gbfs_social_law_generator()
robust_sl = generator.generate_social_law(problem)

print(robust_sl)